In [1]:
import re
import numpy as np

In [2]:
#FILENAME="test.txt"
FILENAME="input.txt"
DIAG=False
#B_MIN = (7,7)
#B_MAX = (27,27)
B_MIN = (200000000000000,200000000000000)
B_MAX = (400000000000000,400000000000000)

In [3]:
def parseFile(my_file_name):    
    with open(my_file_name) as file:
        lines = file.readlines()
    
    dict_points = []
    
    for i,line in enumerate(lines):
        if line[-1]=="\n":
            line=line[:-1]
        if line:
            if DIAG: print("Line: ",line)
            coord,vector = line.split("@")
            x,y,z=[int(t) for t in coord.split(",")]
            dx,dy,dz = [int(t) for t in vector.split(",")]
            
            dict_points+=  [(x,y,z,dx,dy,dz)]
    return dict_points
    


In [4]:
def findPointsIntersect2D(pointA,pointB):
    
    x1,y1,_,dx1,dy1,_ = pointA
    x2,y2=(x1+dx1,y1+dy1)
    
    x3,y3,_,dx3,dy3,_ = pointB
    x4,y4=(x3+dx3,y3+dy3)
    
    int_denom = (x1-x2)*(y3-y4)-(y1-y2)*(x3-x4)
    if int_denom == 0:
        x_int = None
        y_int = None
        
    else:
        x_int_nom =(x1*y2-y1*x2)*(x3-x4)-(x1-x2)*(x3*y4-y3*x4)
        y_int_nom =(x1*y2-y1*x2)*(y3-y4)-(y1-y2)*(x3*y4-y3*x4)
        
        x_int,y_int = (x_int_nom/int_denom,y_int_nom/int_denom)
    
    return (x_int,y_int)

def doPointsIntersectInFuture2D(pointA,pointB,bound1_x,bound1_y,bound2_x,bound2_y):
    x_int,y_int = findPointsIntersect2D(pointA,pointB)
    if x_int is None:
        return False
    elif x_int < bound1_x or x_int > bound2_x or y_int < bound1_y or y_int > bound2_y:
        return False
    else:
        
        
        xA,yA,_,dxA,dyA,_ = pointA
        xB,yB,_,dxB,dyB,_ = pointB
        
        sameA = (np.all(np.sign([x_int-xA,y_int-yA])==np.sign([dxA,dyA])))
        sameB = (np.all(np.sign([x_int-xB,y_int-yB])==np.sign([dxB,dyB])))
        
        
        return (sameA and sameB)

In [5]:
points = parseFile(FILENAME)

In [6]:
res = 0
for i,pA in enumerate(points):
    for j,pB in enumerate(points):
        if i < j:
            if doPointsIntersectInFuture2D(pA,pB,*B_MIN,*B_MAX):
                #print("Inters",pA,pB)                
                res+=1

print("Result:",res)
            

Result: 11246


In [10]:
for i,pA in enumerate(points):
    for j,pB in enumerate(points):
        if i < j:
            _,_,_,dxa,dya,dza = pA
            _,_,_,dxb,dyb,dzb = pB
            if dxa/dxb==dya/dyb and dxa/dxb==dza/dzb:
                print(pA,pB)
print ("Nope")

Nope


In [51]:
import numpy as np
from scipy.optimize import fsolve

p1 = points[0]
p2 = points[1]
p3 = points[2]

p_min = np.minimum.reduce([p1,p2,p3])

p1_tr = p1-p_min
p2_tr = p2-p_min
p3_tr = p3-p_min


def func(x):
    a,b,c,m,n,k,t1,t2,t3=x
    
    return [p1_tr[0]-a+t1*(p1_tr[3]-m),p1_tr[1]-b+t1*(p1_tr[4]-n),p1_tr[2]-c+t1*(p1_tr[5]-k),
            p2_tr[0]-a+t1*(p2_tr[3]-m),p2_tr[1]-b+t1*(p2_tr[4]-n),p2_tr[2]-c+t1*(p2_tr[5]-k),
            p3_tr[0]-a+t1*(p2_tr[3]-m),p3_tr[1]-b+t1*(p3_tr[4]-n),p3_tr[2]-c+t1*(p3_tr[5]-k)]


In [67]:
p1_tr

array([67428652806405, 27197715052215, 85678583542213,             33,
                   11,            281], dtype=int64)

In [73]:
from z3 import *

In [75]:
solv = Solver()

t1 = Int("t1")
t2 = Int("t2")
t3 = Int("t3")
a = Int("a")
b = Int("b")
c = Int("c")
m = Int("m")
n = Int("n")
k = Int("k")

solv.add(t1 > 0)
solv.add(t2 > 0)
solv.add(t3 > 0)

t=[t1,t2,t3]
for i, point in enumerate(points[0:3]):
    x,y,z,dx,dy,dz=point
    solv.add(x+t[i]*dx == a+t[i]*m)
    solv.add(y+t[i]*dy == b+t[i]*n)
    solv.add(z+t[i]*dz == c+t[i]*k)

print(solv.check())
print(solv.model())

[k = 53,
 t1 = 301350904659,
 n = -94,
 t3 = 81593122140,
 m = 330,
 t2 = 824916054430,
 a = 116689373784735,
 b = 348350724549432,
 c = 251559839225936]

In [77]:
res = solv.model()

In [80]:
res[a].as_long()+res[b].as_long()+res[c].as_long()

716599937560103